In [1]:
cd models/research/object_detection

/home/scar3crow/Dropbox/WorkStation-Subrata/python/models/research/object_detection


In [3]:
import numpy as np
import pandas as pd
import cv2
import os
import tqdm
from scipy.io import loadmat

from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder

from PIL import Image
import pytesseract

import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow import keras
from keras import backend as K

from utils import *

from sklearn.model_selection import train_test_split
from keras.callbacks import ModelCheckpoint
from keras.optimizers import Adam
from keras.layers import *

from keras.applications import MobileNetV2
from keras.applications import InceptionResNetV2

from keras.models import Model
from keras.models import model_from_json


In [4]:
# Making a list of image path

inv_directory = '/home/scar3crow/Downloads/INVOICES'  ## 'invoices' is a zip file of jpg images in ...../Downloads 
                                                        
inv_image = ['/home/scar3crow/Downloads/INVOICES/{}'.format(i) for i in os.listdir(inv_directory)] # making the list
inv_image.sort() # Sorting the list

print('Number of images = ', len(inv_image))
inv_image[50]

Number of images =  66


'/home/scar3crow/Downloads/INVOICES/53.jpg'

In [5]:
# Check sizes of exiting images :

# Create a Dataframe with image id and height(row) and width(column):

rows = []
columns = []
image_sl = []
df = pd.DataFrame()

for i in range(len(inv_image)):
    image = cv2.imread(inv_image[i]) ## Loading image
    height, width, _ = image.shape
    rows.append(height)
    columns.append(width)
    image_sl.append(inv_image[i])
    
row_values = pd.Series(rows)
col_values = pd.Series(columns)
image_num = pd.Series(image_sl)


df.insert(loc=0, column='image_serial', value=image_num)
df.insert(loc=1, column='rows', value=row_values)
df.insert(loc=2, column='columns', value=col_values)

df.head(3)

,image_serial,rows,columns
0,/home/scar3crow/Downloads/INVOICES/1.jpg,768,554
1,/home/scar3crow/Downloads/INVOICES/10.jpg,768,554
2,/home/scar3crow/Downloads/INVOICES/11.jpg,768,554


In [70]:
# Loading output of VGG Image Annotation tool and create a dataframe

r_1_data = pd.read_csv('/home/scar3crow/Downloads/invoice_data_1/via_region_data.csv')
r_2_data = pd.read_csv('/home/scar3crow/Downloads/invoice_data_1/2nd_region_data.csv')

r_data = pd.concat([r_1_data, r_2_data], ignore_index=True, sort =False)

r_data.drop(r_data.columns[[1, 2, 3, 4]], axis=1, inplace=True) # reduce unnecessary columns

r_data.rename({'#filename': 'img_id'}, axis=1, inplace=True) # changing column name

r_data.tail(3)

,img_id,region_shape_attributes,region_attributes
445,64.jpg,"{""name"":""rect"",""x"":304,""y"":119,""width"":73,""hei...","{""text"":""po""}"
446,64.jpg,"{""name"":""rect"",""x"":37,""y"":282,""width"":485,""hei...","{""text"":""item_detail""}"
447,64.jpg,"{""name"":""rect"",""x"":460,""y"":500,""width"":66,""hei...","{""text"":""totals""}"


In [71]:
print('Number of classes in diff. categories = ', r_data['img_id'].value_counts())

Number of classes in diff. categories =  9.jpg       7
30.jpg      7
47.jpg      7
54.jpg      7
2.jpg       7
1.jpg       7
59.jpg      7
17.jpg      7
48.jpg      7
12.jpg      7
63.jpg      7
15.jpg      7
20.jpg      7
50.jpg      7
46.jpg      7
60.jpg      7
11.jpg      7
31.jpg      7
10.jpg      7
53.jpg      7
32.jpg      7
3.jpg       7
29.jpg      7
27.jpg      7
55.jpg      7
25.jpg      7
45.jpg      7
62.jpg      7
44.jpg      7
61.jpg      7
           ..
57.jpg      7
19.jpg      7
39.jpg      7
23.jpg      7
4.jpg       7
43.jpg      7
33.jpg      7
42.jpg      7
28.jpg      7
26.jpg      7
36.jpg      7
7.jpg       7
49.jpg      7
58.jpg      7
13.jpg      7
40.jpg      7
16.jpg      7
64.jpg      7
24.jpg      7
5.jpg       7
41.jpg      7
22.jpg      7
21.jpg      7
52.jpg      7
56.jpg      7
6.jpg       7
35.jpg      6
38.jpg      6
37-C.jpg    1
37-B.jpg    1
Name: img_id, Length: 66, dtype: int64


In [75]:
id_a= r_data.index[r_data['img_id'] == '35.jpg']
id_b = r_data.index[r_data['img_id'] == '38.jpg']
id_c = r_data.index[r_data['img_id'] == '37-B.jpg']
id_d = r_data.index[r_data['img_id'] == '37-C.jpg']

print(id_a, id_b, id_c, id_d) 

Int64Index([238, 239, 240, 241, 242, 243], dtype='int64') Int64Index([260, 261, 262, 263, 264, 265], dtype='int64') Int64Index([258], dtype='int64') Int64Index([259], dtype='int64')


In [76]:
r_data[243:261]

,img_id,region_shape_attributes,region_attributes
243,35.jpg,"{""name"":""rect"",""x"":466,""y"":462,""width"":59,""hei...","{""text"":""totals""}"
244,36.jpg,"{""name"":""rect"",""x"":128,""y"":58,""width"":313,""hei...","{""text"":""vendor""}"
245,36.jpg,"{""name"":""rect"",""x"":88,""y"":161,""width"":195,""hei...","{""text"":""buyer""}"
246,36.jpg,"{""name"":""rect"",""x"":321,""y"":160,""width"":103,""he...","{""text"":""invoice""}"
247,36.jpg,"{""name"":""rect"",""x"":323,""y"":177,""width"":117,""he...","{""text"":""inv_date""}"
248,36.jpg,"{""name"":""rect"",""x"":322,""y"":193,""width"":136,""he...","{""text"":""po""}"
249,36.jpg,"{""name"":""rect"",""x"":61,""y"":231,""width"":448,""hei...","{""text"":""item_detail""}"
250,36.jpg,"{""name"":""rect"",""x"":339,""y"":631,""width"":174,""he...","{""text"":""totals""}"
251,37-A.jpg,"{""name"":""rect"",""x"":38,""y"":32,""width"":207,""heig...","{""text"":""vendor""}"
252,37-A.jpg,"{""name"":""rect"",""x"":39,""y"":93,""width"":222,""heig...","{""text"":""buyer""}"


In [77]:

r_data = r_data.drop([258, 259])  # Force delete since there is nothing here and img_id format are different
r_data = r_data.drop([238, 239, 240, 241, 242, 243])
r_data = r_data.drop([260, 261, 262, 263, 264, 265])

r_data = r_data.reset_index(drop=True)

r_data.sort_values(by=['img_id'], ascending=True) # Sorting based on image-id
num_images = r_data["img_id"].nunique() # Find out number of unique images

print('Number of unique images = ', num_images)
r_data.tail()


Number of unique images =  62


,img_id,region_shape_attributes,region_attributes
429,64.jpg,"{""name"":""rect"",""x"":302,""y"":60,""width"":55,""heig...","{""text"":""invoice""}"
430,64.jpg,"{""name"":""rect"",""x"":408,""y"":58,""width"":59,""heig...","{""text"":""inv_date""}"
431,64.jpg,"{""name"":""rect"",""x"":304,""y"":119,""width"":73,""hei...","{""text"":""po""}"
432,64.jpg,"{""name"":""rect"",""x"":37,""y"":282,""width"":485,""hei...","{""text"":""item_detail""}"
433,64.jpg,"{""name"":""rect"",""x"":460,""y"":500,""width"":66,""hei...","{""text"":""totals""}"


In [78]:
# Making a dataframe for Image_id, x, y, width, height and class

x = []
y = []
width = []
height = []
obj_class = []
i_width = 554
i_height = 768


for i in range(len(r_data)):
    
    r_size = r_data.values[i, 1][1:(len(r_data.values[i, 1])-1)]
    r_size_par = r_size.split(",")
    
    x.append(int("".join(filter(str.isdigit, r_size_par[1]))))
    y.append(int("".join(filter(str.isdigit, r_size_par[2]))))
    width.append(int("".join(filter(str.isdigit, r_size_par[3]))))
    height.append(int("".join(filter(str.isdigit, r_size_par[4]))))
    
    r_attribs = r_data.values[i, 2][1:(len(r_data.values[i, 2])-1)]
    r_attribs_par = r_attribs.split(':')[1]
    obj_class.append(r_attribs_par[1:(len(r_attribs_par)-1)])
    
x_values = pd.Series(x)
y_values = pd.Series(y)
width_values = pd.Series(width)
height_values = pd.Series(height)
class_values = pd.Series(obj_class)

r_data.insert(loc=1, column='x', value=x_values)
r_data.insert(loc=2, column='y', value=y_values)
r_data.insert(loc=3, column='width', value=width_values)
r_data.insert(loc=4, column='height', value=height_values)
r_data.insert(loc=5, column='obj_class', value=class_values)

r_data.insert(loc=8, column='image_width', value=i_width)
r_data.insert(loc=9, column='image_height', value=i_height)


r_data.drop(r_data.columns[[6, 7]], axis=1, inplace=True) # reduce unnecessary columns

r_data.rename({'#filename': 'img_id'}, axis=1, inplace=True) # changing column name

r_data.head()

,img_id,x,y,width,height,obj_class,image_width,image_height
0,30.jpg,91,125,394,45,vendor,554,768
1,30.jpg,79,175,426,81,buyer,554,768
2,30.jpg,38,83,136,26,invoice,554,768
3,30.jpg,412,88,76,23,inv_date,554,768
4,30.jpg,94,253,74,19,po,554,768


In [79]:
print('Number of unique images = ', r_data['img_id'].nunique())  # print total no, of unique images

print('Number of classes in diff. categories = ', r_data['obj_class'].value_counts()) 


Number of unique images =  62
Number of classes in diff. categories =  invoice         62
buyer           62
po              62
vendor          62
totals          60
item_detail     60
inv_date        60
total            2
date             2
item_details     1
tem_detail       1
Name: obj_class, dtype: int64


In [80]:
# Correcting above wrong spelling of object classes and rechecking

id_1 = r_data.index[r_data['obj_class'] == 'tem_detail'] # Finding the index
id_2 = r_data.index[r_data['obj_class'] == 'total'] # Finding the index
id_3 = r_data.index[r_data['obj_class'] == 'item_details'] # Finding the index
id_4 = r_data.index[r_data['obj_class'] == 'date']

r_data.at[id_1, 'obj_class'] = 'item_detail' # writing the correct spelling 
r_data.at[id_2, 'obj_class'] = 'totals' # # writing the correct spelling
r_data.at[id_3, 'obj_class'] = 'item_detail' # # writing the correct spelling
r_data.at[id_4, 'obj_class'] = 'inv_date' # # writing the correct spelling

print('Number of unique images = ', r_data['img_id'].nunique())  # print total no, of unique images
print('Number of unique classes = ', r_data['obj_class'].nunique())
print('Number of classes in diff. categories = ', r_data['obj_class'].value_counts()) 


Number of unique images =  62
Number of unique classes =  7
Number of classes in diff. categories =  totals         62
invoice        62
item_detail    62
buyer          62
po             62
vendor         62
inv_date       62
Name: obj_class, dtype: int64


In [86]:
total_df = r_data.groupby('obj_class')
total_obj_df = total_df.get_group('totals').reset_index(drop=True)

In [87]:
total_obj_df.head()

,img_id,x,y,width,height,obj_class,image_width,image_height
0,30.jpg,340,514,193,138,totals,554,768
1,8.jpg,446,451,76,36,totals,554,768
2,7.jpg,446,518,64,28,totals,554,768
3,6.jpg,197,486,322,127,totals,554,768
4,5.jpg,342,472,166,142,totals,554,768


In [96]:
xmax = []
ymax = []

for i in range(len(total_obj_df)):
    
    xmax.append(total_obj_df['x'][i] + total_obj_df['width'][i])   # finding out end of 'totals' boundary box
    ymax.append(total_obj_df['y'][i] + total_obj_df['height'][i])   ## finding out end of 'totals' boundary box
    
max_crop_x = max(xmax)
max_crop_y = max(ymax)

min_crop_x = min(r_data['x'])
min_crop_y = min(r_data['y'])

In [97]:
print(min_crop_x, max_crop_x, min_crop_y, max_crop_y)

6 547 16 679


In [98]:
for i in range(len(inv_image)):
    
    id = inv_image[i].split('/')[-1] # finding the image_id
    image_for_crop = cv2.imread(inv_image[i]) ## Loading image
    crop_train_image = image_for_crop[min_crop_y:max_crop_y, min_crop_x:max_crop_x]
    crop_train_id = '/home/scar3crow/Downloads/train_invoice/detection_train_invoices/' + id
    cv2.imwrite(crop_train_id, crop_train_image)

In [102]:
im =  cv2.imread(inv_image[0])

img = cv2.resize(im,(512, 512))

cv2.imwrite('/home/scar3crow/Downloads/train_invoice/detection_train_invoices/test_2.jpg', img)
print(inv_image[0])

/home/scar3crow/Downloads/INVOICES/1.jpg


In [ ]:
import cv2
def resize_and_letter_box(image, rows, cols):
    """
    Letter box (black bars) a color image (think pan & scan movie shown 
    on widescreen) if not same aspect ratio as specified rows and cols. 
    :param image: numpy.ndarray((image_rows, image_cols, channels), dtype=numpy.uint8)
    :param rows: int rows of letter boxed image returned  
    :param cols: int cols of letter boxed image returned
    :return: numpy.ndarray((rows, cols, channels), dtype=numpy.uint8)
    """
    image_rows, image_cols = image.shape[:2]
    row_ratio = rows / float(image_rows)
    col_ratio = cols / float(image_cols)
    ratio = min(row_ratio, col_ratio)
    image_resized = cv2.resize(image, dsize=(0, 0), fx=ratio, fy=ratio)
    letter_box = np.zeros((int(rows), int(cols), 3))
    row_start = int((letter_box.shape[0] - image_resized.shape[0]) / 2)
    col_start = int((letter_box.shape[1] - image_resized.shape[1]) / 2)
    letter_box[row_start:row_start + image_resized.shape[0], col_start:col_start + image_resized.shape[1]] = image_resized
    return letter_box
